In [10]:
# Importation des librairies pour la requête SQL et l'enregistrement du fichier final :
import requests
import urllib.parse
import sys

import os
import json
from datetime import datetime

In [11]:
# Importation des librairies classiques :
import pandas as pd

In [12]:
# Importation de la configuration des chemins : 
sys.path.append(os.path.abspath(os.path.join("..")))
import config

In [13]:
# Liste des tables disponibles sur le site de l'Agence Européenne à date : 
# NB : Il est possible de compléter cette liste avec les années postérieures 
# (années antérieures non compatibles avec le pre-processing actuel)
table_list = ['co2cars_2021Pv23', 'co2cars_2022Pv25', 'co2cars_2023Pv27']

In [14]:
# Définition de la requête et boucle for pour l'appliquer à tous les noms de table :

records = []

for table in table_list:
  query = f"""
  SELECT DISTINCT [Year] AS Year, Mk, Cn, [M (kg)], [Ewltp (g/km)], Ft, [Ec (cm3)], [Ep (KW)], [Erwltp (g/km)], Fc
  FROM [CO2Emission].[latest].[{table}]
  WHERE Mk IS NOT NULL 
    AND Cn IS NOT NULL 
    AND [M (kg)] IS NOT NULL
    AND [Ewltp (g/km)] IS NOT NULL
    AND Ft IS NOT NULL
    AND [Ec (cm3)] IS NOT NULL
    AND [Ep (KW)] IS NOT NULL
    AND [Erwltp (g/km)] IS NOT NULL
    AND [Year] IS NOT NULL
    AND Fc IS NOT NULL
  """

# Encodage de la requête pour l'inclure dans l'URL :
  encoded_query = urllib.parse.quote(query)

# Initialisation :
  page = 1

# Boucle while pour parcourir toutes les pages de l'API : 
# NB : Celle-ci s'arrête quand il n'y a plus de réponse.

  while True:
    url = f"https://discodata.eea.europa.eu/sql?query={encoded_query}&p={page}&nrOfHits=100000"
    response = requests.get(url)
    data = response.json()
    new_records = data.get("results", [])
    if not new_records:
      break
    records.extend(new_records)
    page += 1

In [15]:
# Transformation en DataFrame :
df = pd.DataFrame(records)
print(df.shape)

# Affichage des premières lignes :
display(df.head())

(300662, 10)


,Year,Mk,Cn,M (kg),Ewltp (g/km),Ft,Ec (cm3),Ep (KW),Erwltp (g/km),Fc
0,2021,ALFA ROMEO,GIULIA,1504,178,PETROL,1995,148,1.2,7.9
1,2021,ALFA ROMEO,GIULIA,1504,179,PETROL,1995,148,1.2,7.9
2,2021,ALFA ROMEO,GIULIA,1504,180,PETROL,1995,148,1.2,8.0
3,2021,ALFA ROMEO,GIULIA,1520,159,PETROL,1995,148,0.8,7.0
4,2021,ALFA ROMEO,GIULIA,1520,160,PETROL,1995,148,0.8,7.0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300662 entries, 0 to 300661
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Year           300662 non-null  int64  
 1   Mk             300662 non-null  object 
 2   Cn             300662 non-null  object 
 3   M (kg)         300662 non-null  int64  
 4   Ewltp (g/km)   300662 non-null  int64  
 5   Ft             300662 non-null  object 
 6   Ec (cm3)       300662 non-null  int64  
 7   Ep (KW)        300662 non-null  int64  
 8   Erwltp (g/km)  300662 non-null  float64
 9   Fc             300662 non-null  float64
dtypes: float64(2), int64(5), object(3)
memory usage: 22.9+ MB


In [17]:
# Définir les chemins vers les dossiers existants :
raw_dir = config.RAW_DIR
metadata_dir = config.METADATA_DIR

# Créer les dossiers s'ils n'existent pas :
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(metadata_dir, exist_ok=True)

# Génération d'un timestamp au format YYYYMMDD_HHMMSS :
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f"DF_Raw_{timestamp}.csv"

# Construction du chemin complet vers le fichier dans le dossier raw existant : 
output_filepath = os.path.join(raw_dir, output_filename)

# Enregistrement du DataFrame dans le fichier avec le nom dynamique : 
df.to_csv(output_filepath, index=False)

# Remplissage du fichier de métadonnées : 
metadata_file = os.path.join(metadata_dir, "metadata.json")
metadata = {"raw_data": output_filepath}
with open(metadata_file, "w") as f:
    json.dump(metadata, f)